In [ ]:
"""Project_Final.ipynb

Automatically generated by Colaboratory.

# Data Preprocessing and Visualisation
"""

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
import sklearn
import random

In [ ]:
from sklearn.utils import shuffle
%matplotlib inline

In [ ]:
from zipfile import ZipFile
with ZipFile('creditcardfraud.zip','r') as zip:
  zip.printdir()
  zip.extractall()

In [ ]:
d=pd.read_csv('creditcard.csv')

In [ ]:
sns.distplot(data['Amount'])

In [ ]:
sns.distplot(data['Time'])

In [ ]:
data.hist(figsize=(20,20))
plt.show()

In [ ]:
sns.jointplot(x= 'Time', y= 'Amount', data= d)

In [ ]:
class0 = d[d['Class']==0]

In [ ]:
len(class0)

In [ ]:
class1 = d[d['Class']==1]

In [ ]:
len(class1)

In [ ]:
class0
temp = shuffle(class0)

In [ ]:
d1 = temp.iloc[:2000,:]

In [ ]:
d1

In [ ]:
frames = [d1, class1]
df_temp = pd.concat(frames)

In [ ]:
df_temp.info()

In [ ]:
df= shuffle(df_temp)

In [ ]:
df.to_csv('creditcardsampling.csv')

In [ ]:
sns.countplot('Class', data=df)

In [ ]:
"""# SMOTE"""

In [ ]:
!pip install --user imblearn

In [ ]:
import imblearn

In [ ]:
from imblearn.over_sampling import  SMOTE
oversample=SMOTE()
X=df.iloc[ : ,:-1]
Y=df.iloc[: , -1]
X,Y=oversample.fit_resample(X,Y)

In [ ]:
X=pd.DataFrame(X)
X.shape

In [ ]:
Y=pd.DataFrame(Y)
Y.head()

In [ ]:
names=['Time','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount','Class']

In [ ]:
data=pd.concat([X,Y],axis=1)

In [ ]:
d=data.values

In [ ]:
data=pd.DataFrame(d,columns=names)

In [ ]:
sns.countplot('Class', data=data)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(data.corr())

In [ ]:
!pip install --user lightgbm

In [ ]:
!pip install --user utils

In [ ]:
import math
import sklearn.preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score , classification_report, confusion_matrix, precision_recall_curve, f1_score, auc

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Class', axis=1), data['Class'], test_size=0.3, random_state=42)

In [ ]:
"""# Feature Scaling"""

In [ ]:
cols= ['V22', 'V24', 'V25', 'V26', 'V27', 'V28']

In [ ]:
scaler = StandardScaler()

In [ ]:
frames= ['Time', 'Amount']

In [ ]:
x= data[frames]

In [ ]:
d_temp = data.drop(frames, axis=1)

In [ ]:
temp_col=scaler.fit_transform(x)

In [ ]:
scaled_col = pd.DataFrame(temp_col, columns=frames)

In [ ]:
scaled_col.head()

In [ ]:
d_scaled = pd.concat([scaled_col, d_temp], axis =1)

In [ ]:
d_scaled.head()

In [ ]:
y = data['Class']

In [ ]:
d_scaled.head()

In [ ]:
"""# Dimensionality Reduction"""

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=7)

In [ ]:
X_temp_reduced = pca.fit_transform(d_scaled)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.explained_variance_

In [ ]:
names=['Time','Amount','Transaction Method','Transaction Id','Location','Type of Card','Bank']

In [ ]:
X_reduced= pd.DataFrame(X_temp_reduced,columns=names)
X_reduced.head()

In [ ]:
Y=d_scaled['Class']

In [ ]:
new_data=pd.concat([X_reduced,Y],axis=1)
new_data.head()
new_data.shape

In [ ]:
new_data.to_csv('finaldata.csv')

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X_reduced, d_scaled['Class'], test_size = 0.30, random_state = 42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
"""# Logistic Regression"""

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred_lr=lr.predict(X_test)
y_pred_lr

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,y_pred_lr))

In [ ]:
#Hyperparamter tuning 
from sklearn.model_selection import GridSearchCV
lr_model = LogisticRegression()
lr_params = {'penalty': ['l1', 'l2'],'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_lr= GridSearchCV(lr_model, param_grid = lr_params)
grid_lr.fit(X_train, y_train)

In [ ]:
grid_lr.best_params_

In [ ]:
y_pred_lr3=grid_lr.predict(X_test)
print(classification_report(y_test,y_pred_lr3))

In [ ]:
"""# Support Vector Machine"""

In [ ]:
from sklearn.svm import SVC
svc=SVC(kernel='rbf')
svc.fit(X_train,y_train)
y_pred_svc=svc.predict(X_test)
y_pred_svc

In [ ]:
print(classification_report(y_test,y_pred_svc))

In [ ]:
print(confusion_matrix(y_test,y_pred_svc))

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [ {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 1, 0.01, 0.0001 ,0.001]}]
grid_search = GridSearchCV(estimator = svc,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
svc_param=SVC(kernel='rbf',gamma=0.01,C=100)
svc_param.fit(X_train,y_train)
y_pred_svc2=svc_param.predict(X_test)
print(classification_report(y_test,y_pred_svc2))

In [ ]:
"""# Decision Tree"""

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)
y_pred_dtree=dtree.predict(X_test)
print(classification_report(y_test,y_pred_dtree))

In [ ]:
print(confusion_matrix(y_test,y_pred_dtree))

In [ ]:
d_tree_param=DecisionTreeClassifier()
tree_parameters={'criterion':['gini','entropy'],'max_depth':list(range(2,4,1)),
                 'min_samples_leaf':list(range(5,7,1))}
grid_tree=GridSearchCV(d_tree_param,tree_parameters)
grid_tree.fit(X_train,y_train)

In [ ]:
y_pred_dtree2=grid_tree.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred_dtree2))

In [ ]:
"""# Random Forest"""

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randomforest=RandomForestClassifier(n_estimators=5)
randomforest.fit(X_train,y_train)
y_pred_rf=randomforest.predict(X_test)
print(confusion_matrix(y_test,y_pred_rf))

In [ ]:
print(classification_report(y_test,y_pred_rf))

In [ ]:
"""# K Nearest Neighbors"""

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
y_pred_knn=knn.predict(X_test)
y_pred_knn

In [ ]:
print(classification_report(y_test,y_pred_knn))

In [ ]:
print(confusion_matrix(y_test,y_pred_knn))

In [ ]:
knn_param=KNeighborsClassifier()
knn_params={"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}
grid_knn=GridSearchCV(knn_param,param_grid=knn_params)
grid_knn.fit(X_train,y_train)
grid_knn.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2)

In [ ]:
knn.fit(X_train,y_train)
pred_knn2 = knn.predict(X_test)

In [ ]:
print('WITH K=3')
print('\n')
print(confusion_matrix(y_test,pred_knn2))
print('\n')
print(classification_report(y_test,pred_knn2))

In [ ]:
"""# XGBoost"""

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred_xg=xgb.predict(X_test)
print(classification_report(y_test,y_pred_xg))

In [ ]:
"""# LGB"""

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data= False)

In [ ]:
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train, free_raw_data= False)

In [ ]:
parameters = {'num_leaves': 2**8,
              'learning_rate': 0.1,
              'is_unbalance': True,
              'min_split_gain': 0.1,
              'min_child_weight': 1,
              'reg_lambda': 1,
              'subsample': 1,
              'objective':'binary',
              #'device': 'gpu', # comment this line if you are not using GPU
              'task': 'train'
              }
num_rounds = 300

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)

In [ ]:
lgb_test = lgb.Dataset(X_test, y_test)

In [ ]:
clf = lgb.train(parameters, lgb_train, num_boost_round=num_rounds)

In [ ]:
y_prob = clf.predict(X_test)
y_pred = sklearn.preprocessing.binarize(np.reshape(y_prob, (-1,1)), threshold= 0.5)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
"""# ROC"""

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score
lg_fpr,lg_tpr,lg_threshold=roc_curve(y_test,y_pred_lr3)
svc_fpr,svc_tpr,svc_threshold=roc_curve(y_test,y_pred_svc2)
dtree_fpr,dtree_tpr,dtree_threshold=roc_curve(y_test,y_pred_dtree2)
rf_fpr,rf_tpr,rf_threshold=roc_curve(y_test,y_pred_rf)
knn_fpr,knn_tpr,rf_threshold=roc_curve(y_test,pred_knn2)
xg_fpr,xg_tpr,xg_threshold=roc_curve(y_test,y_pred_xg)
lgb_fpr,lgb_tpr,lgb_threshold=roc_curve(y_test,y_pred)

In [ ]:
plt.figure(figsize=(15,10))
plt.title("Roc Curve")
plt.plot(lg_fpr,lg_tpr, label='Logistic Regression Classifier Score: {:.4f}'.format(roc_auc_score(y_test, y_pred_lr3)))
plt.plot(knn_fpr,knn_tpr, label='KNears Neighbors Classifier Score: {:.4f}'.format(roc_auc_score(y_test, pred_knn2)))
plt.plot(svc_fpr, svc_tpr, label='Support Vector Classifier Score: {:.4f}'.format(roc_auc_score(y_test, y_pred_svc2)))
plt.plot(dtree_fpr, dtree_tpr, label='Decision Tree Classifier Score: {:.4f}'.format(roc_auc_score(y_test, y_pred_dtree2)))
plt.plot(rf_fpr,rf_tpr, label='Random Forest Classifier Score: {:.4f}'.format(roc_auc_score(y_test, y_pred_rf)))
plt.plot(xg_fpr,xg_tpr, label='XGBoost Classifier Score: {:.4f}'.format(roc_auc_score(y_test, y_pred_xg)))
plt.plot(lgb_fpr,lgb_tpr, label='Light Gradient Boosting Classifier Score: {:.4f}'.format(roc_auc_score(y_test, y_pred)))
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.legend()
plt.show()